# Construyamos un App con Gen AI ✨

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/EnriqueVilchezL/ai_workshop_2025_gen_ai_session_2/blob/main/incomplete.ipynb)

En este taller vamos a aprender a utilizar grandes modelos de lenguaje de manera local, y a construir una pequeña aplicación encima de ese servicio.

## 1. Instalar dependencias

### 1.1. Ollama

Primero, vamos a instalar **Ollama** en la máquina que nos ofrece Google Colab.

**Ollama** es una herramienta de código abierto que permite ejecutar modelos de lenguaje de gran tamaño (LLM) directamente en sus computadoras, sin necesidad de conexión a Internet ni servicios en la nube. Esto la convierte en una opción ideal para desarrolladores, investigadores y empresas que buscan mantener el control total sobre sus datos y reducir costos asociados a APIs externas.

Para más información pueden visitar [Ollama](https://ollama.com/).

En un sistema Linux, Ollama se puede instalar con:

In [ ]:
!sudo apt -q update
!sudo apt -q install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

### 1.2. Python

Ahora, vamos a instalar los paquetes necesarios para ejecutar nuestra aplicación en python. Las principales bibliotecas que vamos a usar son:

1. Langchain: Permite usar LLMs de manera sencilla, ya sea de servicios en la nube o local. Para más información pueden visitar [Langchain](https://www.langchain.com/).

2. Gradio: Permite montar interfaces web para modelos de inteligencia artificial de forma sencilla. Para más información pueden visitar [Gradio](https://www.gradio.app/).

In [ ]:
%pip install langchain langchain_community gradio ollama pypdf

 ## 2. Iniciando con LLMs locales

 Ahora, vamos a utilizar LLMs de forma local, descargando modelos en la máquina de Google Colab y aprovechando la GPU gratuita que nos ofrece.

### 2.1. Correr ollama

Primero, vamos a correr ollama. Ollama funciona como un servidor que escucha en un puerto de su computadora local, en el cual va a ejecutar LLMs. Normalmente, ollama se corre desde una terminal con el comando "ollama serve", pero como no tenemos acceso a una terminal para la máquina de Google Colab, vamos a utilizar la biblioteca de [Ollama en python](https://github.com/ollama/ollama-python).

In [ ]:
import threading
import subprocess
import time

def run_ollama_server() -> None:
    subprocess.Popen(["ollama", "serve"])

threading.Thread(target=run_ollama_server).start()
time.sleep(5)

Ahora, vamos a descargar un modelo de la nube a la máquina local.

In [ ]:
import ollama

ollama.pull("llama3.1:8b")

Podemos probar el modelo que acabamos de descargar.

In [ ]:
import asyncio
from ollama import AsyncClient

async def chat(message: str):
    message = {'role': 'user', 'content': message}
    response = await AsyncClient().chat(model='llama3.1:8b', messages=[message])
    print(response.message.content)

await chat('Why is the sky blue?')

### 2.2. Hacer un chat con langchain y ollama

Podríamos usar la función "chat" de ollama para interactuar con el modelo, sin embargo, langchain da muchas facilidades para controlar más el modelo y el proovedor.

Esto funciona. Sin embargo, hay dos particularidades de los LLMs que vale la pena conocer:

#### **Plantillas**

Los LLMs no interpretan directamente los mensajes “tal cual” se ingresan. Internamente, los mensajes se organizan mediante plantillas de texto estructuradas, que ayudan al modelo a comprender:
- Quién está hablando (user, assistant, system)
- Cuál es el contexto
- Dónde debe empezar a generar su respuesta

Estas plantillas varían según el modelo y pueden incluir delimitadores especiales, instrucciones o formato específico. Generalmente se introducen tokens especiales antes de cada interacción. Esto le permite al modelo entender el contexto en el que interactúa.

Un ejemplo de esto es:
```
<|system|>
Eres una IA útil.
<|user|>
¿Cuál es la capital de Francia?
<|assistant|>
```

Afortunadamente, Langchain puede manejar estos detalles por nosotros.

#### **Memoria**

Es importante entender que, a pesar de parecer inteligentes y conversacionales, los modelos de lenguaje no tienen memoria a largo plazo. Esto significa que:
- No recuerdan conversaciones anteriores a menos que se les proporcione explícitamente el historial.
- Cada interacción con el modelo es independiente, a menos que se incluya el contexto anterior en el mismo prompt.
- Toda la “memoria” del modelo está limitada al contenido del mensaje que se le envía en ese momento.

Cuando se mantiene una conversación con un modelo como LLaMA, GPT o Mistral, el modelo no guarda información sobre lo que se dijo antes. Si se le pregunta algo en la ronda 2 que depende de lo que ocurrió en la ronda 1, solo podrá responder correctamente si se le incluye el historial de forma explícita.

Para dar la ilusión de memoria en aplicaciones prácticas, se pasa todo el historial de conversación como parte del nuevo mensaje (prompt chaining).

### 2.3. Texto dinámico

Si corremos ollama en una terminal, vamos a ver que es mucho más bonito que el chat que hemos montado hasta el momento. Esto se debe a que los LLMs producen token por token, y no oraciones completas cada vez que mandamos una entrada, por lo que se puede imprimir los tokens conforme los va generando.

Langchain ofrece una interfaz sencilla para hacer esto.

## 3. Aplicación con Gradio

Lo que hemos hecho hasta ahora está muy bien, pero quizá queremos mostrarle nuestro trabajo a nuestros amigos... Gradio ofrece una interfaz muy sencilla para montar una aplicación web que sirva para LLMs.

## 4. Análisis de archivos

Una posible aplicación interesante de los LLMs es utilizarlos para analizar archivos como PDFs o TXTs. Con gradio es sumamente sencillo integrar esto a la interfaz web.

## 5. Conclusiones

Este taller ha demostrado que es totalmente posible correr modelos de lenguaje grandes (LLMs) **de forma local**, sin depender de servicios externos en la nube. Gracias a herramientas como **Ollama**, **Langchain** y **Gradio**, puedes construir rápidamente aplicaciones conversacionales personalizadas, privadas y adaptables.

Al finalizar este notebook, deberías tener un conocimiento básico sobre:

- Cómo instalar y correr LLMs localmente usando Ollama.
- Cómo crear interacciones personalizadas y memorias conversacionales con Langchain.
- Cómo construir una interfaz de usuario sencilla con Gradio.
- Cómo extender la funcionalidad del modelo para trabajar con archivos (como PDFs).

Este es solo el comienzo: pueden adaptar esta arquitectura para hacer agentes inteligentes (ver [Curso de agentes](https://huggingface.co/learn/agents-course/en/unit0/introduction)), asistentes para tu trabajo, chatbots educativos, o incluso sistemas RAG (ver [Taller de RAG con uv](https://github.com/EnriqueVilchezL/ai_workshop_2025_gen_ai_session.git) y [Taller de RAG con poetry](https://github.com/Antonio-Tresol/ai_workshop_2025_gen_ai_session.git)).

---

Gracias por participar en el taller 🙌 ¡A seguir creando con IA generativa!